## Step 1: Remove Useless columns
In this step, we load all **Shopify** order files and combine them into a single DataFrame. We then select only valuable columns for our models.
Columns we removed are:

In [1]:
import pandas as pd
import os

RAW_DATA_PATH = os.path.join("..", "resources", "data", "raw", "clv")
PROCESSED_DATA_PATH = os.path.join("..", "resources", "data", "processed", "clv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")

# File paths
order_files = [
    os.path.join(RAW_DATA_PATH, "orders_export_2.csv"),
    os.path.join(RAW_DATA_PATH, "orders_export_3.csv"),
    os.path.join(RAW_DATA_PATH, "orders_export_1.csv"),
]

##########step 1 all orders in one dataframe##########
# all orders tables in one dataframe
raw_orders_df = pd.concat([pd.read_csv(file) for file in order_files], ignore_index=True)

################step 2 Keep only usefull columns###############

orders_cleaned_columns = raw_orders_df[["Name", "Email", # identifiers
                                        "Total", # order total
                                        "Discount Amount", 
                                        "Created at", # order creation date
                                        "Lineitem quantity",
                                        "Lineitem price",
                                        "Refunded Amount"]]

orders_cleaned_columns.to_csv(output_path, index=False)
orders_cleaned_columns.head()

C:\Users\AbdulS\AppData\Local\Temp\ipykernel_9580\3696433823.py:17: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_orders_df = pd.concat([pd.read_csv(file) for file in order_files], ignore_index=True)
C:\Users\AbdulS\AppData\Local\Temp\ipykernel_9580\3696433823.py:17: DtypeWarning: Columns (44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_orders_df = pd.concat([pd.read_csv(file) for file in order_files], ignore_index=True)


,Name,Email,Total,Discount Amount,Created at,Lineitem quantity,Lineitem price,Refunded Amount
0,31607,tinekesnelting@outlook.com,90.11,0.0,2024-02-26 15:41:51 +0100,1,29.95,0.0
1,31607,tinekesnelting@outlook.com,NaN,NaN,2024-02-26 15:41:51 +0100,1,34.95,NaN
2,31607,tinekesnelting@outlook.com,NaN,NaN,2024-02-26 15:41:51 +0100,1,29.95,NaN
3,31607,tinekesnelting@outlook.com,NaN,NaN,2024-02-26 15:41:51 +0100,1,-4.74,NaN
4,31602,marianvanweert@gmail.com,124.39,0.0,2024-02-26 15:41:50 +0100,2,27.95,0.0


## Drop all columns where email is missing
Columns with a missing email have an uknown customer, there is no value in predicting the clv for unknown customers

In [2]:
input_path = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")
df = pd.read_csv(input_path)
df = df[df["Email"].notna() & (df["Email"].str.strip() != "")]
df.reset_index(drop=True, inplace=True)
df.to_csv(output_path, index=False)

## Step 2: Combine Multiple Rows Into One Per Order
Shopify splits a single order into multiple rows if it contains multiple unique items. This step identifies such rows using the 'Email' and 'Name' (which is the id of an order) fields, and merges them by combining their line items into a single list. Additionally, duplicate orders are filtered to keep only the fulfilled ones. The result is a dataset where each order occupies one row.

In [ ]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
orders_df = pd.read_csv(orders_file)

# Normalize email
orders_df["Email"] = orders_df["Email"].str.lower()

# Create structured line item object per row including price
orders_df["Lineitem_object"] = orders_df.apply(
    lambda row: {
        "qty": int(row["Lineitem quantity"]),
        "price": float(row["Lineitem price"])
    },
    axis=1
)

# Group all line items per order
lineitems_grouped = orders_df.groupby(["Name", "Email"])["Lineitem_object"].agg(list).reset_index()

# Keep only the first row per order for metadata
first_rows = orders_df.drop_duplicates(subset=["Name", "Email"], keep="first")

# Merge grouped line items back into first rows
merged_orders = pd.merge(
    first_rows.drop(columns=["Lineitem sku", "Lineitem quantity", "Lineitem price", "Lineitem_object"]),
    lineitems_grouped,
    on=["Name", "Email"],
    how="left"
)

# Rename column for clarity
merged_orders = merged_orders.rename(columns={"Lineitem_object": "Lineitems"})

# Save the final one-row-per-order file
merged_orders.to_csv(output_path, index=False)
###################################

# Count number of orders per customer
order_counts = merged_orders.groupby("Email").size().reset_index(name="num_orders")

# Count how many customers fall into each order count group
order_distribution = order_counts["num_orders"].value_counts().sort_index()

# Print the result
print("Number of customers by number of orders:")
print(order_distribution)


Number of customers by number of orders:
num_orders
1     12700
2      3371
3      1272
4       485
5       255
6        99
7        43
8        26
9        10
10        5
11        4
12        4
13        4
16        2
27        1
39        1
Name: count, dtype: int64


## One row per customer
We need a dataset where a customer has exactly one row. At the moment, a customer has multiple rows in the orders table, each representing a separate order. To fix this, we:  
- Aggregated each customer's past behavior up to a cutoff date (3 months before the last order), including total spend, order count, average order value, recency, and customer lifetime.  
- Calculated monthly features like spend and order count per month.  
- Computed the CLV target as total spend in the 3 months after the cutoff.  
- Merged everything into one row per customer for modeling.

In [47]:
input_path = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "customers_with_flat_orders.csv")

orders_df = pd.read_csv(input_path)
orders_df["Email"] = orders_df["Email"].str.lower()
orders_df["Paid at"] = pd.to_datetime(orders_df["Paid at"], utc=True, errors="coerce")
orders_df = orders_df.dropna(subset=["Paid at"])

# Add YearMonth for monthly grouping
orders_df["YearMonth"] = orders_df["Paid at"].dt.to_period("M").astype(str)

# Get static customer info from their last order
latest_info = orders_df.sort_values("Paid at").drop_duplicates("Email", keep="last")[
    ["Email", "Shipping City", "Shipping Country"]
].rename(columns={"Shipping City": "City", "Shipping Country": "Country"})

# Set cutoff date for CLV target
cutoff_date = orders_df["Paid at"].max() - pd.DateOffset(months=3)

# Split past and future orders
orders_df = orders_df.sort_values(by=["Email", "Paid at"])
past_orders = orders_df[orders_df["Paid at"] < cutoff_date]
future_orders = orders_df[orders_df["Paid at"] >= cutoff_date]

# Aggregate past behavior
past_summary = past_orders.groupby("Email").agg(
    total_spent_before=("Total", "sum"),
    avg_order_value=("Total", "mean"),
    order_count_before=("Total", "count"),
    total_discount_before=("Discount Amount", "sum"),
    first_order_date=("Paid at", "min"),
    last_order_date=("Paid at", "max")
).reset_index()

# Time-based features
past_summary["recency_days"] = (cutoff_date - past_summary["last_order_date"]).dt.days
past_summary["customer_lifetime_days"] = (past_summary["last_order_date"] - past_summary["first_order_date"]).dt.days
past_summary["days_since_first_order"] = (cutoff_date - past_summary["first_order_date"]).dt.days

# Monthly behavior features
monthly_summary = past_orders.groupby(["Email", "YearMonth"]).agg(
    orders_per_month=("Name", "count"),
    amount_per_month=("Total", "sum"),
    discount_per_month=("Discount Amount", "sum")
).reset_index()

monthly_pivot = monthly_summary.pivot(index="Email", columns="YearMonth")
monthly_pivot.columns = [f"{col[1]}_{col[0]}" for col in monthly_pivot.columns]
monthly_pivot = monthly_pivot.reset_index()

# Extra features
monthly_spend_cols = [col for col in monthly_pivot.columns if "amount_per_month" in col]
monthly_pivot["monthly_spend_std"] = monthly_pivot[monthly_spend_cols].std(axis=1)
monthly_pivot["months_active"] = monthly_pivot[monthly_spend_cols].notna().sum(axis=1)

# CLV target (next 3 months)
future_clv = future_orders.groupby("Email")["Total"].sum().reset_index()
future_clv = future_clv.rename(columns={"Total": "clv_next_3_months"})

# Merge all
clv_dataset = pd.merge(past_summary, monthly_pivot, on="Email", how="left")
clv_dataset = pd.merge(clv_dataset, future_clv, on="Email", how="left")
clv_dataset = pd.merge(clv_dataset, latest_info, on="Email", how="left")
clv_dataset["clv_next_3_months"] = clv_dataset["clv_next_3_months"].fillna(0)
clv_dataset.to_csv(output_path, index=False)



C:\Users\AbdulS\AppData\Local\Temp\ipykernel_34324\608406288.py:10: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  orders_df["YearMonth"] = orders_df["Paid at"].dt.to_period("M").astype(str)


## let's do the same for klaviyo
We only will select columns that we presume can improve the model performance.


In [44]:

KLAVIYO_FILE = os.path.join(RAW_DATA_PATH, "Klaviyo_everyone_email.csv")
OUTPUT_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, "klaviyo_cleaned_columns.csv")

customers_df = pd.read_csv(KLAVIYO_FILE)
# for now we will only keep the columns these columns if the model still needs more columns we can add them later
customers_cleaned_columns = customers_df[[
    "Email",
    "Email Marketing Consent",
    "Email Marketing Consent Timestamp",
    "Email Suppressions Timestamp",
    "First Active",
    "Last Active",
    "Profile Created On",
    "Last Open",
    "Last Click"
]]
customers_cleaned_columns.to_csv(OUTPUT_DATA_PATH, index=False)

C:\Users\AbdulS\AppData\Local\Temp\ipykernel_34324\1345683836.py:4: DtypeWarning: Columns (6,65,66,68,72) have mixed types. Specify dtype option on import or set low_memory=False.
  customers_df = pd.read_csv(KLAVIYO_FILE)


## Step 3: Merge Klaviyo Customer Data
We merge the cleaned Shopify order data with customer profile data from Klaviyo using the 'Email' field. This ensures that we capture both customers who have made purchases and those who have not. The result is a dataset that combines behavioral (order-based) and demographic (profile-based) information.

In [45]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "customers_with_flat_orders.csv")
klaviyo_file = os.path.join(PROCESSED_DATA_PATH, "klaviyo_cleaned_columns.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data.csv")

orders_df = pd.read_csv(orders_file)
customers_df = pd.read_csv(klaviyo_file)

merged_df = pd.merge(
    customers_df,
    orders_df,
    on="Email",
    how="outer",
    suffixes=("", "shopify")
)

merged_df.to_csv(output_path, index=False)
print(f"Merged file with all customers saved to: {output_path}")
merged_df.head()


C:\Users\AbdulS\AppData\Local\Temp\ipykernel_34324\1194478784.py:5: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  orders_df = pd.read_csv(orders_file)


Merged file with all customers saved to: ..\resources\data\processed\clv\orders_with_customer_data.csv


,Email,Email Marketing Consent,Email Marketing Consent Timestamp,Email Suppressions Timestamp,First Active,Last Active,Profile Created On,Last Open,Last Click,Name,...,order_4_Lineitems,order_5_Lineitems,order_6_Lineitems,order_7_Lineitems,order_8_Lineitems,order_9_Lineitems,order_10_Lineitems,order_11_Lineitems,order_12_Lineitems,order_13_Lineitems
0,'wolters.bente@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MU23418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.chocolade-macumba@icloud.com,UNSUBSCRIBED,2024-09-27 10:21:43 CEST,"[""2024-09-27 10:21:43 CEST""]",2024-03-18 16:53:43,2025-01-27 12:07:21,2024-03-18 16:53:44,2024-09-28 10:51:03,2024-04-13 21:19:20,MU18184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01docile_ream@icloud.com,NEVER_SUBSCRIBED,NaN,"[""2024-06-26 22:42:54 CEST""]",2024-06-12 19:22:29,2024-08-13 10:29:39,2024-06-12 19:22:33,2024-07-15 10:43:02,2024-06-19 22:54:21,MU20659,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01wendy@planet.nl,SUBSCRIBED,2024-08-13 09:41:08 CEST,[],2024-08-13 09:40:54,2025-03-07 16:13:26,2024-08-13 09:41:08,2025-03-07 16:13:26,NaN,MU23443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0303.nicole@gmail.com,NEVER_SUBSCRIBED,NaN,[],2023-01-22 12:17:13,2024-02-16 15:58:35,2023-01-22 12:17:14,2023-11-10 19:06:24,NaN,13225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 5: Create Monthly Spend and Order Features
Here, we engineer new features for modeling. For each customer, we compute the number of orders and the amount spent in each month. These features are crucial for understanding customer behavior over time and are especially useful for predicting future value.

In [ ]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "monthly_customer_stats.csv")

orders_df = pd.read_csv(orders_file, low_memory=False)
orders_df["Paid at"] = pd.to_datetime(orders_df["Paid at"], utc=True, errors="coerce")
orders_df = orders_df.dropna(subset=["Paid at"])
orders_df["YearMonth"] = orders_df["Paid at"].dt.strftime("%Y_%m")

monthly_summary = orders_df.groupby(["Email", "YearMonth"]).agg(
    orders_per_month=("Name", "count"),
    amount_per_month=("Total", "sum")
).reset_index()

order_counts = monthly_summary.pivot(index="Email", columns="YearMonth", values="orders_per_month")
amounts_spent = monthly_summary.pivot(index="Email", columns="YearMonth", values="amount_per_month")

order_counts = order_counts.reindex(sorted(order_counts.columns), axis=1)
amounts_spent = amounts_spent.reindex(sorted(amounts_spent.columns), axis=1)

order_counts.columns = [f"orders_in_{col}" for col in order_counts.columns]
amounts_spent.columns = [f"amount_spent_{col}" for col in amounts_spent.columns]

monthly_stats = pd.concat([order_counts, amounts_spent], axis=1)
monthly_stats["total_amount_spent"] = orders_df.groupby("Email")["Total"].sum()

monthly_stats = monthly_stats.reset_index()
monthly_stats.to_csv(output_file, index=False)
print(f"Monthly customer stats saved to: {output_file}")
monthly_stats.head()


## Step 5: Merge Monthly Features Into Full Dataset
In this final preprocessing step, we merge the engineered monthly features into the combined Klaviyo + Shopify dataset. This gives us one complete table that includes customer profile data, order history, and time-based behavioral features, ready for training a Customer Lifetime Value prediction model.

In [ ]:
orders_customers_file = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data.csv")
monthly_stats_file = os.path.join(PROCESSED_DATA_PATH, "monthly_customer_stats.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "final_merged_data.csv")

orders_customers_df = pd.read_csv(orders_customers_file)
monthly_stats_df = pd.read_csv(monthly_stats_file)

final_df = pd.merge(
    orders_customers_df,
    monthly_stats_df,
    on="Email",
    how="left"
)

final_df.to_csv(output_file, index=False)
print(f"Final merged dataset saved to: {output_file}")
final_df.head()


### Step 6 – Final Feature Selection and Aggregation

In this step, we finalized our customer-level dataset by cleaning, transforming, and aggregating features to prepare them for modeling.

#### Column Filtering
We removed over 70 columns that were either:
- Sensitive (e.g. names, addresses, phone numbers)
- Duplicated or irrelevant for modeling
- High-cardinality categorical columns unlikely to provide predictive power
- System-generated metadata or unused campaign tracking codes

#### Boolean Normalization
We identified object-type columns that semantically represented booleans (e.g., `InWelcomeFlow`, `InCheckoutAbandonmentFlow`, `Accepts Marketing`) and explicitly converted them to `True`/`False`, filling missing values as `False`.

#### Location Cleanup
Missing values in the `Country` column were assumed to be “Netherlands”, as that is the primary market. Missing `City`, `Last Source`, and `Campaign` fields were filled with `"unknown"` or `"(direct)"` where applicable.

#### Date Feature Engineering
We engineered customer timeline features based on:
- First and last activity
- First and last purchase
- Days since email marketing consent

This resulted in numerical columns like `days_since_first_active` and `customer_lifetime_days`, giving the model a richer temporal understanding of each customer.

#### Email Marketing Consent
Instead of treating `Email Marketing Consent Timestamp` as a raw date, we extracted:
- `email_marketing_optin`: binary indicator if consent was ever given
- `days_since_email_optin`: numeric time since consent (or -1 if not given)

This allowed us to convert a sparse timestamp into usable model features.

#### Lineitem Breakdown
Each row in the dataset originally represented a customer order and contained a `Lineitems` field — a list of `{sku: quantity}` dictionaries.
We:
- Counted total quantities (`total_amount_line_items_bought`)
- Extracted the first and second unique SKUs bought, plus their quantities (`first_item_sku`, `first_item_qty`, etc.)

This allowed us to preserve product-level information in a compact way, without introducing excessive sparsity.

####  Aggregation to Customer Level
As multiple orders could exist for a single customer, we grouped the dataset by `Email`, and:
- Summed all numeric fields (e.g. amount spent, quantity, days)
- Took the first occurrence of remaining non-numeric fields

This gave us one row per customer, suitable for training a model to predict their future value.


In [ ]:
import pandas as pd
import ast
from datetime import datetime
import os

input_file = os.path.join(PROCESSED_DATA_PATH, "final_merged_data.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "clv_model_data.csv")

# Load data
df = pd.read_csv(input_file)

# Drop irrelevant or sensitive columns
columns_to_drop = [
    'First Name', 'Last Name', 'Phone Number', 'Address', 'Address 2', 'Locale',
    'Locale: Country', 'Organization', 'Title', 'Birthday', 'birthday', '$birthday_source', 'Billing Company',
    'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country', 'Billing Province Name',
    'Shipping City', 'Shipping Province', 'Shipping Country', 'Shipping Province Name',
    'Email Suppressions', 'Email Suppressions Timestamp',
    'Email List Suppressions', 'Email List Suppressions Timestamp', 'Email List Suppressions Reasons',
    'Initial Referring Domain', 'Search Keyword', 'Initial Source Referrer',
    'Created at', 'Cancelled at', 'Payment Method', 'Payment ID', 'Id', 'Tags', 'Risk Level', 'Source',
    'Subtotal', 'Taxes', 'Total', 'Discount Code', 'Refunded Amount', 'Duties',
    'Lineitem quantity', 'Lineitem price', 'Lineitem sku', 'Lineitem discount',
    'Paid at', 'Fulfilled at', 'Financial Status', 'Accepts Marketing_order', 'undefined',
    'Profile Created On', 'Date Added', 'Last Open', 'Last Click',
    'Device ID', 'Location', 'Employee',
    'SMS Marketing Consent', 'SMS Marketing Consent Timestamp',
    'SMS Transactional Consent', 'SMS Transactional Consent Timestamp',
    'Total Customer Lifetime Value', 'Historic Customer Lifetime Value',
    'Historic Number Of Orders', 'Initial Source Content', 'Last Source Term',
    'Last Source First Page', 'Last Source Medium', 'Last Source Content', 
    '$timezone', 'CollectionPreference',
    'Fulfillment Status', 'Name', 'ZPD: Situation', 'VIP', 'UTM Source',
    'UTM Medium', 'ResendFulfilledOrderFlow', 'Survey 01/23',
    'T-LwHoyq: Swimwear: yes', 'UTM Campaign', 'UTM Content',
    'Last Source Referrer', 'Initial Source First Page', 'Initial Source Term',
    'Expected Date Of Next Order'
]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Convert object-type booleans
bool_like_cols = ['InCheckoutAbandonmentFlow', 'InFulfilledOrderFlow', 'InWelcomeFlow', 'Accepts Marketing']
for col in bool_like_cols:
    df[col] = df[col].fillna(False).astype(bool)

# Fill defaults for important string fields
df["Country"] = df["Country"].fillna("Netherlands")
df["City"] = df["City"].fillna("unknown")
df["Last Source Campaign"] = df["Last Source Campaign"].fillna("unknown")
df["Last Source"] = df["Last Source"].fillna("unknown")
df["Initial Source Campaign"] = df["Initial Source Campaign"].fillna("(direct)")

# Parse Lineitems
df["Lineitems"] = df["Lineitems"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Convert dates
date_cols = ["First Active", "Last Active", "First Purchase Date", "Last Purchase Date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

reference_date = datetime(2025, 4, 1)

# Feature engineering from dates
df["email_marketing_optin"] = df["Email Marketing Consent Timestamp"].notna().astype(int)
df["days_since_email_optin"] = (
    datetime(2025, 4, 1) - pd.to_datetime(df["Email Marketing Consent Timestamp"], errors="coerce")
).dt.days

# Fill missing with 0 or -1 depending on what makes more sense
df["days_since_email_optin"] = df["days_since_email_optin"].fillna(-1)
df.drop(columns=["Email Marketing Consent Timestamp"], inplace=True)

df["days_since_first_active"] = (reference_date - df["First Active"]).dt.days
df["days_since_last_active"] = (reference_date - df["Last Active"]).dt.days
df["days_since_first_purchase"] = (reference_date - df["First Purchase Date"]).dt.days
df["days_since_last_purchase"] = (reference_date - df["Last Purchase Date"]).dt.days
df["customer_lifetime_days"] = (df["Last Purchase Date"] - df["First Purchase Date"]).dt.days

# Drop original datetime columns
df.drop(columns=date_cols, inplace=True)

# Count total quantity of items in Lineitems
df["order_items_count"] = df["Lineitems"].apply(
    lambda items: sum(qty for item in items for qty in item.values()) if isinstance(items, list) else 0
)

# Aggregate to one row per customer
numeric_cols = df.select_dtypes(include='number').columns.tolist()
grouped_df = df.groupby("Email").agg({
    **{col: 'sum' for col in numeric_cols},
    **{col: 'first' for col in df.columns if col not in numeric_cols and col != "Email"}
}).reset_index()

# Rename engineered columns
grouped_df.rename(columns={
    "Discount Amount": "total_discount_amount",
    "order_items_count": "total_amount_line_items_bought"
}, inplace=True)

# ✨ Extract first, second, third item SKU and qty
def extract_item_and_qty(items, index):
    if isinstance(items, list) and len(items) > index:
        item = list(items[index].items())[0]
        return item[0], item[1]
    return None, 0

for i, label in enumerate(["first", "second"]):
    grouped_df[f"{label}_item_sku"] = grouped_df["Lineitems"].apply(lambda items: extract_item_and_qty(items, i)[0])
    grouped_df[f"{label}_item_qty"] = grouped_df["Lineitems"].apply(lambda items: extract_item_and_qty(items, i)[1])

# Save final dataset
grouped_df.to_csv(output_file, index=False)
print(f"Saved cleaned customer-level dataset to: {output_file}")


## Step 7: Create the Target Variable (CLV_next_3_months)

To train a model that predicts customer lifetime value (CLV) for the next 3 months, we first need to define our target.

### What we’re predicting:
We are summing the total amount a customer spent in the months of January, February, and March 2025. This becomes the column `CLV_next_3_months`, which will serve as our prediction target.

### Why this approach:
- Simple and interpretable
- Allows the model to focus on short-term CLV
- Aligns with business needs like quarterly revenue forecasting


In [ ]:
import pandas as pd
input_file = os.path.join(PROCESSED_DATA_PATH, "clv_model_data.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "clv_model_data_with_target.csv")

df = pd.read_csv(input_file)

# Define the target variable
target_months = ["amount_spent_2025_01", "amount_spent_2025_02", "amount_spent_2025_03"]
df["CLV_next_3_months"] = df[target_months].sum(axis=1)

# Drop future columns from features to avoid leakage
df.drop(columns=target_months, inplace=True)

df.to_csv(output_file, index=False)
print("Target variable created and future columns dropped.")


## Step 8: Pre-process Features for Modeling

Before training a machine learning model, we need to prepare our feature set.

### What we do:
- Drop identifier columns like `Email` which are not predictive
- Convert boolean columns into integer format (`True` → `1`, `False` → `0`)
- One-hot encode object/categorical columns for compatibility with XGBoost
- Replace special characters in column names to avoid parsing errors
- Fill remaining missing values with `0` to avoid training issues

### Let's show the correlation heatmap

In [ ]:
import seaborn as sns

# Load data with target
input_file = os.path.join(PROCESSED_DATA_PATH, "clv_model_data_with_target.csv")
df = pd.read_csv(input_file)
import matplotlib.pyplot as plt

# Only use numeric columns for correlation
corr = df.select_dtypes(include='number').corr()

plt.figure(figsize=(16, 12))
sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap of Numeric Features")
plt.tight_layout()
plt.show()


In [ ]:
# Load data with target
input_file = os.path.join(PROCESSED_DATA_PATH, "clv_model_data_with_target.csv")
df = pd.read_csv(input_file)
# Drop identifiers
df = df.drop(columns=["Email"])

# Encode boolean columns as integers
bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)

# Fill NaNs
df.fillna(0, inplace=True)

# Separate features and target
X = df.drop(columns=["CLV_next_3_months"])
y = df["CLV_next_3_months"].squeeze()

# Encode object/categorical columns using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Replace ALL non-alphanumeric characters with underscores
X.columns = X.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)

## Step 9: Train/Test Split

To evaluate how well our model generalizes to unseen customers, we split our dataset into training and testing sets.

We use an 80/20 split, a common practice in supervised learning.


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
X_train_np = X_train.to_numpy()
y_train_np = np.asarray(y_train).ravel()  # 1D target
X_test_np = X_test.to_numpy()
y_test_np = np.asarray(y_test).ravel()

## Step 10: Baseline for CLV Prediction
Zero Spending: Predict 0 for all customers.

Global Average: Predict average of 3-month spend across all customers.

Customer-Specific Monthly Average × 3: Predict Total Amount Spent / # active months × 3.

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# True values
y_true = y_test_np

# 1. Zero spending baseline
y_zero = np.zeros_like(y_true)

# 2. Global average baseline
global_avg = y_train_np.mean()
y_global = np.full_like(y_true, global_avg)

# 3. Customer-specific monthly avg × 3 (only if available)
# For now, we simulate a per-customer average from training set
# You can replace this logic with actual per-customer monthly avg if available
y_personal_avg = np.full_like(y_true, (y_train_np.mean() / 3) * 3)

# Define a helper function
def print_metrics(name, y_pred):
    print(f"🔹 {name}")
    print("  RMSE:", round(mean_squared_error(y_true, y_pred), 2))
    print("  MAE :", round(mean_absolute_error(y_true, y_pred), 2))
    print("  R²  :", round(r2_score(y_true, y_pred), 4))

# Show results
print_metrics("Zero Spending Baseline", y_zero)
print_metrics("Global Average Baseline", y_global)
print_metrics("Customer Avg x 3 Baseline", y_personal_avg)

## Step 10: Choose, Train, and Evaluate Model

We will use **XGBoost Regressor**, a powerful and efficient model for tabular data.

### Why XGBoost:
- Performs well on structured data
- Handles missing values internally
- Provides feature importance insights
- Scales well for large datasets

We will train the model and evaluate its performance using:
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score (explained variance)


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import time

start = time.time()
# model = XGBRegressor(random_state=42)
model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.5,
    reg_lambda=1,
    random_state=42
)
model.fit(X_train_np, y_train_np)

print(f"Training completed in {time.time() - start:.2f} seconds")

# Predict and evaluate
y_pred = model.predict(X_test_np)
root_mean_squared_error(y_test_np, y_pred)
RMSE_XGBoost = root_mean_squared_error(y_test_np, y_pred)
MAE_XGBoost = mean_absolute_error(y_test_np, y_pred)
R2_XGBoost = r2_score(y_test_np, y_pred)
print("RMSE:", RMSE_XGBoost)
print("MAE:", MAE_XGBoost)
print("R² Score:", R2_XGBoost)

Great the model gets just above 85% accuracy.
## let's plot the results

In [ ]:
import matplotlib.pyplot as plt

# Model performance metrics
rmse = RMSE_XGBoost
mae = MAE_XGBoost
r2 = R2_XGBoost

# Labels and values
metrics = ['RMSE', 'MAE', 'R² Score']
values = [rmse, mae, r2]

# Create bar plot
plt.figure(figsize=(8, 5))
plt.bar(metrics, values, color='skyblue')
plt.title("Model Performance Metrics")
plt.ylabel("Score")
plt.ylim(0, max(values) + 5)

# Annotate values on bars
for i, v in enumerate(values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Predicted vs Actual Scatter Plot
# Shows how well predictions align with true values.
plt.figure(figsize=(6, 6))
plt.scatter(y_test_np, y_pred, alpha=0.3)
plt.plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 'r--')
plt.xlabel("Actual CLV")
plt.ylabel("Predicted CLV")
plt.title("Predicted vs Actual CLV")
plt.grid(True)
plt.tight_layout()
plt.show()

## Let's do the same with Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import time

# Initialize and train model LINEAR REGRESSION
start = time.time()
# lr_model = LinearRegression()
# lr_model.fit(X_train_np, y_train_np)


# Initialize and train model TUNED LINEAR REGRESSION
elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5)  # l1_ratio = 0.5 means equal L1 and L2 penalty
elastic_model.fit(X_train_np, y_train_np)
print(f"Training completed in {time.time() - start:.2f} seconds")

# Predict and evaluate
y_pred_lr = elastic_model.predict(X_test_np)

rmse_lr = root_mean_squared_error(y_test_np, y_pred_lr)
mae_lr = mean_absolute_error(y_test_np, y_pred_lr)
r2_lr = r2_score(y_test_np, y_pred_lr)

print("Linear Regression Results:")
print("RMSE:", rmse_lr)
print("MAE:", mae_lr)
print("R² Score:", r2_lr)

In [ ]:
import matplotlib.pyplot as plt

# Model performance metrics
rmse = rmse_lr
mae = mae_lr
r2 = r2_lr

# Labels and values
metrics = ['RMSE', 'MAE', 'R² Score']
values = [rmse, mae, r2]

# Create bar plot
plt.figure(figsize=(8, 5))
plt.bar(metrics, values, color='skyblue')
plt.title("Model Performance Metrics")
plt.ylabel("Score")
plt.ylim(0, max(values) + 5)

# Annotate values on bars
for i, v in enumerate(values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Predicted vs Actual Scatter Plot
# Shows how well predictions align with true values.
plt.figure(figsize=(6, 6))
plt.scatter(y_test_np, y_pred_lr, alpha=0.3)
plt.plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 'r--')
plt.xlabel("Actual CLV")
plt.ylabel("Predicted CLV")
plt.title("Predicted vs Actual CLV")
plt.grid(True)
plt.tight_layout()
plt.show()